## Calling SNPs 

### Using samtools mpileup 

#### First I need to convert my SAM alignment files to BAM files. Then sort and index an la 

In [10]:
!pwd

/Users/laurelnave-powers/Desktop/GitHub/Laurel-genes/data


In [12]:
cd /Applications/bioinfo/samtools1.11/bin

/Applications/bioinfo/samtools1.11/bin


In [14]:
! ls

ace2sam               md5fa                 samtools
blast2sam.pl          md5sum-lite           samtools.pl
bowtie2sam.pl         novo2sam.pl           seq_cache_populate.pl
export2sam.pl         plot-ampliconstats    soap2sam.pl
interpolate_sam.pl    plot-bamstats         wgsim
maq2sam-long          psl2sam.pl            wgsim_eval.pl
maq2sam-short         sam2vcf.pl            zoom2sam.pl


In [2]:
!samtools


Program: samtools (Tools for alignments in the SAM format)
Version: 1.11 (using htslib 1.11)

Usage:   samtools <command> [options]

Commands:
  -- Indexing
     dict           create a sequence dictionary file
     faidx          index/extract FASTA
     fqidx          index/extract FASTQ
     index          index alignment

  -- Editing
     calmd          recalculate MD/NM tags and '=' bases
     fixmate        fix mate information
     reheader       replace BAM header
     targetcut      cut fosmid regions (for fosmid pool only)
     addreplacerg   adds or replaces RG tags
     markdup        mark duplicates
     ampliconclip   clip oligos from the end of reads

  -- File operations
     collate        shuffle and group alignments by name
     cat            concatenate BAMs
     merge          merge sorted alignments
     mpileup        multi-way pileup
     sort           sort alignment file
     split          splits a file by read group
     quickcheck     quickly check if SA

In [5]:
!/Applications/bioinfo/samtools1.11/bin/samtools \
samtools view -bS ../analyses/HIPP-DR.sam \
| samtools sort -o ../analyses/HIPP-DR.bam \
samtools index ../analyses/HIPP-DR.bam

Usage: samtools sort [options...] [in.bam]
Options:
  -l INT     Set compression level, from 0 (uncompressed) to 9 (best)
  -u         Output uncompressed data (equivalent to -l 0)
  -m INT     Set maximum memory per thread; suffix K/M/G recognized [768M]
  -M         Use minimiser for clustering unaligned/unplaced reads
  -K INT     Kmer size to use for minimiser [20]
  -n         Sort by read name (not compatible with samtools index command)
  -t TAG     Sort by value of TAG. Uses position as secondary index (or read name if -n is set)
  -o FILE    Write final output to FILE rather than standard output
  -T PREFIX  Write temporary files to PREFIX.nnnn.bam
  --no-PG    do not add a PG line
[main] unrecognized command 'samtools'
      --input-fmt-option OPT[=VAL]
               Specify a single input file format option in the form
               of OPTION or OPTION=VALUE
  -O, --output-fmt FORMAT[,OPT[=VAL]]...
               Specify output format (SAM, BAM, CRAM)
      --output-fmt-op

#### Then I need to mpileup the la 

In [ ]:
!/Applications/bioinfo/samtools1.11/bin/samtools \
samtools mpileup --VCF --no-BAQ \
--fasta-ref Danio_rerio.GRCz11.dna_sm.toplevel.fa.gz \
../analyses/HIPP-DR.bam \
-o ../analyses/HIPP-DR.vcf.gz